# Kalman Filter Multi-step Prediction

## Import and Initial Setup
Set directory to load log files from.

In [ ]:
import os
import pandas as pd
import numpy as np
import rospkg

# Create a RosPack object
rospack = rospkg.RosPack()

# Get the path to the package this script is in
package_path = rospack.get_path('hri_predict_ros')

# Define the path to the logs directory
log_dir = os.path.join(package_path, 'logs')
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Specify the path to the rosbag files and the gui data
bag_dir = os.path.join(log_dir, 'bag')
gui_dir = os.path.join(log_dir, 'gui_data')
npz_dir = os.path.join(log_dir, 'npz')

# Define the path to the plots directory
plot_dir = os.path.join(package_path, 'plots')
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)